In [1]:
from google.colab import userdata
import os

api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

os.environ["OPENAI_API_BASE"] = "https://api.aimlapi.com/v1"

In [2]:
!pip install -q langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.6 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableSequence, RunnablePassthrough
from langchain.memory import ConversationBufferMemory

##Create a Memory Object

In [4]:
# Simple in-memory conversation
memory = ConversationBufferMemory(memory_key="chat_history")

/tmp/ipython-input-182861835.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


##Define Prompt with Memory

In [5]:
# Prompt template that uses memory
prompt = ChatPromptTemplate.from_template(
    "The following is a conversation so far:\n{chat_history}\n"
    "User: {user_input}\nAI:"
)

##Define the Chain

In [6]:
model = ChatOpenAI(model="gpt-4o-mini")

# Use RunnablePassthrough to attach memory
chain = RunnableSequence(
    first=RunnablePassthrough(additional_keys=["chat_history", "user_input"]),
    middle=[prompt, model],
    last=StrOutputParser()
)

##Run a Conversational Loop

In [8]:
# --- Memory ---
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=False)

# --- Prompt ---
prompt = ChatPromptTemplate.from_template(
    "The following is a conversation so far:\n{chat_history}\n"
    "User: {user_input}\nAI:"
)

# --- Model ---
model = ChatOpenAI(model="gpt-4o-mini")

# --- Chain ---
chain = RunnableSequence(
    first=RunnablePassthrough(additional_keys=["chat_history", "user_input"]),
    middle=[prompt, model],
    last=StrOutputParser()
)

# --- Multi-turn conversation ---
# First turn
memory_vars = memory.load_memory_variables({})
user_input1 = "Hello!"
input1 = {
    "user_input": user_input1,
    "chat_history": memory_vars["chat_history"]
}
response1 = chain.invoke(input1)
print("AI:", response1)

# Save to memory
memory.save_context({"user_input": user_input1}, {"output": response1})

# Second turn
memory_vars = memory.load_memory_variables({})
user_input2 = "How are you?"
input2 = {
    "user_input": user_input2,
    "chat_history": memory_vars["chat_history"]
}
response2 = chain.invoke(input2)
print("AI:", response2)

AI: Hello! How can I assist you today?
AI: I'm just a program, but I'm here and ready to help you! How are you?
